## MQO Ceteris Paribus

### Lendo os Dados

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from numpy import dot
from numpy.linalg import inv

variables = {0: "age",
             1: "female",
             3: "education",
             4: "earnings",
             5: "hours",
             6: "week"}


dataset = (pd.read_csv("./data/cps09mar.txt", sep = "\t", header=None)
            .rename(index=str, columns=variables)
           .assign(earnings=lambda df: df["earnings"] / (df["hours"] * df["week"]),
                   tenure=lambda df: df["age"] - df["education"] - 6)
           .query("earnings>0")
           .loc[:, list(variables.values()) + ["tenure"]]
          )

print(dataset.head().to_html())
# dataset.head()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>age</th>
      <th>female</th>
      <th>education</th>
      <th>earnings</th>
      <th>hours</th>
      <th>week</th>
      <th>tenure</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>52</td>
      <td>0</td>
      <td>12</td>
      <td>62.393162</td>
      <td>45</td>
      <td>52</td>
      <td>34</td>
    </tr>
    <tr>
      <th>1</th>
      <td>38</td>
      <td>0</td>
      <td>18</td>
      <td>21.367521</td>
      <td>45</td>
      <td>52</td>
      <td>14</td>
    </tr>
    <tr>
      <th>2</th>
      <td>38</td>
      <td>0</td>
      <td>14</td>
      <td>15.686275</td>
      <td>40</td>
      <td>51</td>
      <td>18</td>
    </tr>
    <tr>
      <th>3</th>
      <td>41</td>
      <td>1</td>
      <td>13</td>
      <td>22.596154</td>
      <td>40</td>
      <td>52</td>
      <td>22</td>
    </tr>
    <tr>
      <th>4</th>
      <td>42</td>
 

## OLS Model

In [2]:
import statsmodels.formula.api as smf

results = smf.ols('np.log(earnings) ~ education + age + female', data=dataset).fit()

results.summary().tables[1]
# print(results.summary().tables[1].as_html())

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,1.1501,0.016,71.436,0.000,1.119,1.182
education,0.1082,0.001,114.390,0.000,0.106,0.110
age,0.0095,0.000,42.276,0.000,0.009,0.010
female,-0.2629,0.005,-50.165,0.000,-0.273,-0.253


In [3]:
X = (dataset
     .loc[:, ["education", "age", "female"]]
     .assign(intercept=1))

y = (dataset
     .assign(log_earnings= lambda df: np.log(df["earnings"]))
     .loc[:, ["log_earnings"]])

In [4]:
def OLS_estimator(X, y):
    variables = X.columns
    beta_hat = inv(X.T.dot(X)).dot(X.T).dot(y).reshape(-1,)
    return pd.DataFrame({"variables": variables, "beta_hat":beta_hat})

def OLS_predictor(X, y):
    variables = X.columns
    beta_hat = inv(X.T.dot(X)).dot(X.T).dot(y).reshape(-1,)
    return X.values.dot(beta_hat)

OLS_estimator(X, y)

,beta_hat,variables
0,0.108156,education
1,0.009540,age
2,-0.262894,female
3,1.150116,intercept


### Least Squares Residuals

In [5]:
X = (dataset
     .loc[:, ["education", "age", "female"]]
     .assign(intercept=1))

y = (dataset
     .assign(log_earnings= lambda df: np.log(df["earnings"]))
     .loc[:, ["log_earnings"]])

beta_hat = OLS_estimator(X, y).beta_hat
y_hat = X.values.dot(beta_hat)
e_hat = y.values.reshape(-1,) - y_hat

In [6]:
np.round(e_hat.mean(), 6) # mean zero

-0.0

In [7]:
np.round(X.values.T.dot(e_hat), 6) # no correlation with X

array([-0., -0., -0., -0.])

## Projection Matirx

In [8]:
sample = dataset.sample(1000)

X = (sample
     .loc[:, ["education", "age", "female"]]
     .assign(intercept=1)
     .values)

y = (sample
     .assign(log_earnings= lambda df: np.log(df["earnings"]))
     .loc[:, ["log_earnings"]]
     .values)

beta_hat = inv(X.T.dot(X)).dot(X.T).dot(y)


P = X.dot(inv(X.T.dot(X))).dot(X.T) # this is huge. Watch out!!! (NxN)

In [9]:
y_hat1 = P.dot(y) 
y_hat2 = X.dot(beta_hat)

In [10]:
np.all(np.isclose(y_hat2, y_hat1, 1e-6)) # both are the same

True

## Ortogonal Projection Matirx

In [11]:
M = np.eye(P.shape[0]) - P

In [12]:
np.round(M.dot(X), 6)

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-0., -0., -0., -0.]])

In [13]:
e_hat1 = M.dot(y)
e_hat2 = y - y_hat1
e_hat1.mean(), e_hat2.mean()

(1.361755153084232e-14, 1.3500311979441903e-14)

In [14]:
np.all(np.isclose(e_hat1, e_hat2, 1e-6)) # both are the same

True

## Regression on Components

In [15]:
def regress(y, X):
    return inv(X.T.dot(X)).dot(X.T).dot(y)

X = (dataset
     .loc[:, ["education", "age", "female"]]
     .assign(intercept=1))

y = (dataset
     .assign(log_earnings= lambda df: np.log(df["earnings"]))
     .loc[:, ["log_earnings"]]
     .values)

X1 = X.loc[:, ["intercept", "age", "female"]].values
X2 = X.loc[:, ["education"]].values

In [16]:
beta1_tilde = regress(y, X1)
y1_hat_tilde = X1.dot(beta1_tilde)
e1_tilde = y - y1_hat_tilde

beta2_tilde = regress(X2, X1)
X2_hat_tilde = X1.dot(beta2_tilde)
X2e_tilde = X2 - X2_hat_tilde

beta2_hat = regress(e1_tilde, X2e_tilde)

beta2_hat # coef for education

array([[0.10815617]])

In [17]:
regress(y, X.values)

array([[ 0.10815617],
       [ 0.00954044],
       [-0.26289372],
       [ 1.15011621]])